# UMAP Parametric
#### Required packages: 
+ umap 0.5.0
+ !pip install llvmlite>=0.34.0
+ pynndescent # for the neural network embedding 

#### Main Resource : 
+ https://colab.research.google.com/drive/1lpdCy7HkC5TRI9LfUtIHBBW8oRO86Nvi?usp=sharing#scrollTo=NUEqcHlvXyKL


In [24]:
import numpy as np
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import itertools as it 
import sklearn
from sklearn.cluster import SpectralClustering
from sklearn.preprocessing import normalize

import pandas as pd
import pickle
import plotly
import plotly.graph_objs as pgo
import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.io as pio

import seaborn as sns
import umap
from pynndescent import NNDescent
import numpy as np
from tensorflow.keras.datasets import mnist
from sklearn.utils import check_random_state
from umap.umap_ import fuzzy_simplicial_set

%matplotlib inline

## FUNCTIONS

In [17]:
# -------------------------------------
# the original function by Felix: 
# -------------------------------------

'''
Random Walk Operator with restart probability.
Return Matrix.
''' 
def rnd_walk_matrix2(A, r, a, num_nodes):

    num = 1*num_nodes
    n = num_nodes
    factor = float((1-a)/n) # = 0 if alpha = 1.0 

    E = np.multiply(factor,np.ones([n,n]))              # prepare 2nd scaling term
    A_tele = np.multiply(a,A) + E  #     print(A_tele)
    M = normalize(A_tele, norm='l1', axis=0)                                 # column wise normalized MArkov matrix

    # mixture of Markov chains
    del A_tele
    del E

    U = np.identity(n,dtype=int) 
    H = (1-r)*M
    H1 = np.subtract(U,H)
    del U
    del M
    del H    

    W = r*np.linalg.inv(H1)   

    return W


# -------------------------------------
# rwr modified (as used in this script) 
# -------------------------------------

def rwr_matrix(A, r): 
    n = len(A) # take the whole network into account = all nodes are seed nodes
    factor = 1 
    a = 1 # maximum freedom to jump anytime to any node within the network
    
    E = np.multiply(factor,np.ones([n,n])) # Matrix with all 1 --> to 
    A_tele = np.multiply(a,A) + E  #     print(A_tele)
    M = normalize(A_tele, norm='l1', axis=0)                                 # column wise normalized MArkov matrix

    # mixture of Markov chains
    del A_tele
    del E

    U = np.identity(n,dtype=int) 
    H = (1-r)*M
    H1 = np.subtract(U,H)
    del U
    del M
    del H    

    W = r*np.linalg.inv(H1)   

    return W


# -------------------------------------
# rwr simulated (just for testing, output = path from random walker)
# -------------------------------------

def random_walk_simple(start_v, number_of_steps):

    for step in range(1, number_of_steps):

        start_vertex = start_v # index of vertex to start from
        visited_vertices = {} # Dictionary that associate nodes with the amount of times it was visited   

        path = [start_vertex] # Store and print path  

        counter = 0 # Restart the cycle
        for counter in range(1, number_of_steps): 
            vertex_neighbors = [n for n in G.neighbors(start_vertex)] # get adjacent nodes
            probability = [] # Set probability of going to a neighbour is uniform
            probability = probability + [1./len(vertex_neighbors)] * len(vertex_neighbors)

            start_vertex = np.random.choice(vertex_neighbors, p=probability) # Choose a vertex from the vertex neighborhood to start the next random walk

            if start_vertex in visited_vertices: # Accumulate the amount of times each vertex is visited
                visited_vertices[start_vertex] += 1
            else:
                visited_vertices[start_vertex] = 1

            path.append(start_vertex) # Append to path

        # mostvisited = sorted(visited_vertices, key = visited_vertices.get,reverse = True) # Organize the vertex list in most visited decrescent order
        # print("Path: ", path)
        # print("Most visited nodes: ", mostvisited[:10]) # Separate the top 10 most visited vertex
        
        return path

    
# -------------------------------------
# plotting functions 
# -------------------------------------   

def draw_node_degree(G, scalef):
    #x = 20
    #ring_frac = np.sqrt((x-1.)/x)
    #ring_frac = (x-1.)/x

    l_size = {}
    for node in G.nodes():
        k = nx.degree(G, node)
        R = scalef * (1 + k**1.1) 

        l_size[node] = R
        
    return l_size


def draw_node_degree_3D(G, scalef):
    x = 3
    ring_frac = (x-1.)/x

    deg = dict(G.degree())
    
    d_size = {}
    for i in G.nodes():
        for k,v in deg.items():
            if i == k:
                R = scalef * (1+v**0.9)
                r = ring_frac * R
                d_size[i] = R
    
    return d_size 


def embed_umap_2D(Matrix, n_neighbors, spread, min_dist, metric='cosine'):
    n_components = 2 # for 2D

    U = umap.UMAP(
        n_neighbors = n_neighbors,
        spread = spread,
        min_dist = min_dist,
        n_components = n_components,
        metric = metric)
    embed = U.fit_transform(Matrix)
    
    return embed


def get_posG_2D(l_nodes, embed):
    posG = {}
    cc = 0
    for entz in l_nodes:
        # posG[str(entz)] = (embed[cc,0],embed[cc,1])
        posG[entz] = (embed[cc,0],embed[cc,1])
        cc += 1

    return posG


def get_posG_3D(l_genes, embed):
    posG = {}
    cc = 0
    for entz in l_genes:
        posG[entz] = (embed[cc,0],embed[cc,1],embed[cc,2])
        cc += 1
    
    return posG


def color_nodes_from_dict(G, d_to_be_coloured, color_method):

    # Colouring
    colour_groups = set(d_to_be_coloured.values())
    colour_count = len(colour_groups)
    pal = sns.color_palette('YlOrRd', colour_count)
    palette = pal.as_hex()

    d_colourgroups = {}
    for n in colour_groups:
        d_colourgroups[n] = [k for k in d_to_be_coloured.keys() if d_to_be_coloured[k] == n]
        
    d_colourgroups_sorted = {key:d_colourgroups[key] for key in sorted(d_colourgroups.keys())}

    d_val_col = {}
    for idx,val in enumerate(d_colourgroups_sorted):
        for ix,v in enumerate(palette):
            if idx == ix:
                d_val_col[val] = v

    d_node_colour = {}
    for y in d_to_be_coloured.items(): # y[0] = node id, y[1] = val
        for x in d_val_col.items(): # x[0] = val, x[1] = (col,col,col)
            if x[0] == y[1]:
                d_node_colour[y[0]]=x[1]

    # SORT dict based on G.nodes
    d_node_colour_sorted = dict([(key, d_node_colour[key]) for key in G.nodes()])
    l_col = list(d_node_colour_sorted.values())
    colours = l_col

    return colours

def color_clusters_from_dict(G, d_to_be_coloured, color_method):

    # Colouring
    colour_groups = set(d_to_be_coloured.values())
    colour_count = len(colour_groups)
    pal = sns.color_palette('Spectral', colour_count)
    palette = pal.as_hex()

    d_colourgroups = {}
    for n in colour_groups:
        d_colourgroups[n] = [k for k in d_to_be_coloured.keys() if d_to_be_coloured[k] == n]
        
    d_colourgroups_sorted = {key:d_colourgroups[key] for key in sorted(d_colourgroups.keys())}

    d_val_col = {}
    for idx,val in enumerate(d_colourgroups_sorted):
        for ix,v in enumerate(palette):
            if idx == ix:
                d_val_col[val] = v

    d_node_colour = {}
    for y in d_to_be_coloured.items(): # y[0] = node id, y[1] = val
        for x in d_val_col.items(): # x[0] = val, x[1] = (col,col,col)
            if x[0] == y[1]:
                d_node_colour[y[0]]=x[1]

    # SORT dict based on G.nodes
    d_node_colour_sorted = dict([(key, d_node_colour[key]) for key in G.nodes()])
    l_col = list(d_node_colour_sorted.values())
    colours = l_col
    
    return colours


def embed_umap_3D(Matrix, n_neighbors, spread, min_dist, metric='cosine'):
    n_components = 3 # for 3D

    U_3d = umap.UMAP(
        n_neighbors = n_neighbors,
        spread = spread,
        min_dist = min_dist,
        n_components = n_components,
        metric = metric)
    embed = U_3d.fit_transform(Matrix)
    
    return embed


def get_posG_3D(l_genes, embed):
    posG = {}
    cc = 0
    for entz in l_genes:
        posG[entz] = (embed[cc,0],embed[cc,1],embed[cc,2])
        cc += 1
    
    return posG


def get_trace_nodes_3D(posG, info_list, color_list, size):

    key_list=list(posG.keys())
    trace = pgo.Scatter3d(x=[posG[key_list[i]][0] for i in range(len(key_list))],
                           y=[posG[key_list[i]][1] for i in range(len(key_list))],
                           z=[posG[key_list[i]][2] for i in range(len(key_list))],
                           mode = 'markers',
                           text = info_list,
                           hoverinfo = 'text',
                           #textposition='middle center',
                           marker = dict(
                color = color_list,
                size = size,
                symbol = 'circle',
                line = dict(width = 1.0,
                        color = color_list)
            ),
        )
    
    return trace


def get_trace_edges_3D(G, posG, color_list, opac = 0.2):
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in G.edges():
        x0, y0, z0 = posG[edge[0]]
        x1, y1, z1 = posG[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)
        edge_z.append(z0)
        edge_z.append(z1)
        edge_z.append(None)

    trace_edges = pgo.Scatter3d(
                        x = edge_x, 
                        y = edge_y, 
                        z = edge_z,
                        mode = 'lines', hoverinfo='none',
                        line = dict(width = 0.5, color = color_list),
                        opacity = opac
                )
    
    return trace_edges


def color_nodes_same(G, list_color_nodes, color):

    d_col = {}
    for node in list_color_nodes:
            d_col[node] = color
      
    return d_col 

In [16]:
# GRAPH input 
organism = 'Human'

G = nx.read_edgelist('input/ppi_elist.txt',data=False)
# d_ent_sym, d_sym_ent = genent2sym()

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

df_gene_sym = pd.read_csv('output_csv/DF_gene_symbol_Human.csv', index_col=0)
l_features = list((df_gene_sym.to_dict()).values())

posG_entrez = []
for k in G.nodes():
    posG_entrez.append(k)
    
    
features_MF = pd.read_csv('output_csv/Features_GO_MolFunc_Dataframe_Human.csv', index_col=0)
feat_genes = list(features_MF.index)
    
    
A_graph = nx.adjacency_matrix(G)
A = A_graph.toarray()

In [20]:
r = 0.8 

struct_rwr_matrix = rwr_matrix(A,r)

df_struct = pd.DataFrame(struct_rwr_matrix, columns = list(G.nodes()), index=list(G.nodes()))

In [ ]:
# INPUT FOR EMBEDDING ≠ Distance Matrix, but instead FEATURE MATRIX

#DM = df_struct
#feature='structural'

DM = df_func
feature='functional'

# set gene list (= G.nodes())
genes = []
for i in DM.index:
    genes.append(str(i))

genes_rest = [] 
for g in G.nodes():
    if g not in genes:
        genes_rest.append(g)

# Parametric UMAP 2D

In [ ]:
from umap.parametric_umap import ParametricUMAP
from pynndescent import NNDescent

from umap.umap_ import fuzzy_simplicial_set

In [ ]:
X = np.array(features_MF)

In [ ]:
X = X.reshape(15165,3744,1,1)

In [ ]:
X.shape

-------

## Part 1 : Building the Graph 
#### Build nearest neighbor's graph

In [21]:
# load dataset
#(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
#X = train_images.reshape((60000, 28, 28, 1))
#X.shape

In [ ]:
# number of trees in random projection forest
n_trees = 5 + int(round((X.shape[0]) ** 0.5 / 20.0))

# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(X.shape[0]))))

# distance metric
metric="euclidean"

# number of neighbors for computing k-neighbor graph
n_neighbors = 10

# get nearest neighbors
nnd = NNDescent(
    X.reshape((len(X), np.product(np.shape(X)[1:]))),
    n_neighbors=n_neighbors,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
# get indices and distances
knn_indices, knn_dists = nnd.neighbor_graph

In [ ]:
# nearest neighbors and distances for each point in the dataset
np.shape(knn_indices), np.shape(knn_dists)

+ build fuzzy simplicial

In [ ]:
from sklearn.utils import check_random_state
from umap.umap_ import fuzzy_simplicial_set

In [ ]:
# get indices and distances
knn_indices, knn_dists = nnd.neighbor_graph
random_state = check_random_state(None)

In [ ]:
# build fuzzy_simplicial_set
umap_graph, sigmas, rhos = fuzzy_simplicial_set(
    X = X,
    n_neighbors = n_neighbors,
    metric = metric,
    random_state = random_state,
    knn_indices= knn_indices,
    knn_dists = knn_dists,
)

In [ ]:
umap_graph

+ embedding the graph 

In [ ]:
import tensorflow as tf

In [ ]:
n_components = 2 # number of latent dimensions
dims = X.shape[1:]
encoder = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=dims),
    tf.keras.layers.Conv2D(
        filters=64, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
    ),
    tf.keras.layers.Conv2D(
        filters=128, kernel_size=3, strides=(2, 2), activation="relu", padding="same"
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=512, activation="relu"),
    tf.keras.layers.Dense(units=512, activation="relu"),
    tf.keras.layers.Dense(units=n_components),
])
encoder.summary()

+ create batch iterator > iteration of batches of edges and nearste-neighbors

In [ ]:
from umap.parametric_umap import construct_edge_dataset

In [ ]:
# n_epochs isused to compute epochs_per_sample, which, in non-parametric UMAP, 
# is the total number of epochs to optimize embeddings over. The computed value 
# epochs_per_sample, is the number of epochs  each edge is optimized over 
# (higher probability = more epochs).
n_epochs = 200 

batch_size = 1000 # iterate over batches of 1000 edges

In [ ]:
# get tensorflow dataset of edges 
(
    edge_dataset,
    batch_size,
    n_edges,
    head,
    tail,
    edge_weight,
) = construct_edge_dataset(
    X,
    umap_graph,
    n_epochs,
    batch_size,
    parametric_embedding = True,
    parametric_reconstruction = False,
)

In [ ]:
(sample_edge_to_x, sample_edge_from_x), _ =  next(iter(edge_dataset))
sample_edge_to_x.shape

+ compute loss over a batch 

In [ ]:
from umap.parametric_umap import umap_loss
from umap.umap_ import find_ab_params

In [ ]:
min_dist = 0.1 # controls how tightly UMAP is allowed to pack points together (0 is more)
_a, _b = find_ab_params(1.0, min_dist)

+ define the loss function

In [ ]:
negative_sample_rate = 5 # how many negative samples to train on per edge. 

In [ ]:
# grab z for the edge
embedding_to = encoder.predict(sample_edge_to_x)
embedding_from = encoder.predict(sample_edge_from_x)

In [ ]:
# get negative samples by randomly shuffling the batch
embedding_neg_to = tf.repeat(embedding_to, negative_sample_rate, axis=0)
repeat_neg = tf.repeat(embedding_from, negative_sample_rate, axis=0)
embedding_neg_from = tf.gather(
    repeat_neg, tf.random.shuffle(tf.range(tf.shape(repeat_neg)[0]))
)

In [ ]:
# Euclidean distances between samples (and negative samples)
distance_embedding = tf.concat(
    [
        tf.norm(embedding_to - embedding_from, axis=1),
        tf.norm(embedding_neg_to - embedding_neg_from, axis=1),
    ],
    axis=0,
)

In [ ]:
def convert_distance_to_probability(distances, a=1.0, b=1.0):
    return 1.0 / (1.0 + a * distances ** (2 * b))

# convert probabilities to distances
probabilities_distance = convert_distance_to_probability(
    distance_embedding, _a, _b
)

In [ ]:
# set true probabilities based on negative sampling
probabilities_graph = tf.concat(
    [tf.ones(batch_size), tf.zeros(batch_size * negative_sample_rate)], axis=0,
)

In [ ]:
def compute_cross_entropy(
    probabilities_graph, probabilities_distance, EPS=1e-4, repulsion_strength=1.0
):
    # cross entropy
    attraction_term = -probabilities_graph * tf.math.log(
        tf.clip_by_value(probabilities_distance, EPS, 1.0)
    )
    repellant_term = (
        -(1.0 - probabilities_graph)
        * tf.math.log(tf.clip_by_value(1.0 - probabilities_distance, EPS, 1.0))
        * repulsion_strength
    )

    # balance the expected losses between atrraction and repel
    CE = attraction_term + repellant_term
    return attraction_term, repellant_term, CE

In [ ]:
# compute cross entropy
(attraction_loss, repellant_loss, ce_loss) = compute_cross_entropy(
    probabilities_graph,
    probabilities_distance,
)

In [ ]:
from umap.parametric_umap import umap_loss 

umap_loss_fn = umap_loss(
    batch_size,
    negative_sample_rate,
    _a,
    _b,
    edge_weight,
    parametric_embedding = True
)

+ define and compile the Keras model

In [ ]:
# define the inputs
to_x = tf.keras.layers.Input(shape=dims, name="to_x")
from_x = tf.keras.layers.Input(shape=dims, name="from_x")
inputs = [to_x, from_x]

# parametric embedding
embedding_to = encoder(to_x)
embedding_from = encoder(from_x)

# concatenate to/from projections for loss computation
embedding_to_from = tf.concat([embedding_to, embedding_from], axis=1)
embedding_to_from = tf.keras.layers.Lambda(lambda x: x, name="umap")(
    embedding_to_from
)
outputs = {'umap': embedding_to_from}

# create model
parametric_model = tf.keras.Model(inputs=inputs, outputs=outputs,)

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-3)

In [ ]:
parametric_model.compile(
    optimizer=optimizer, loss=umap_loss_fn
)

+ fit model

In [ ]:
steps_per_epoch = int(
    n_edges / batch_size / 5
)

In [ ]:
# create embedding
history = parametric_model.fit(
    edge_dataset,
    epochs=1,
    steps_per_epoch=steps_per_epoch,
    max_queue_size=100,
)

In [ ]:
plt.plot(history.history['loss'])

+ take look at final embeddings 

In [ ]:
# Project data
z = encoder.predict(X)

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(10, 8))
sc = ax.scatter(
    z[:, 0],
    z[:, 1],
    c=train_labels,
    cmap="tab10",
    s=0.1,
    alpha=0.5,
    rasterized=True,
)
ax.axis('equal')
ax.set_title("UMAP embeddings", fontsize=20)

In [8]:
# number of trees in random projection forest
n_trees = 5 + int(round((X.shape[0]) ** 0.5 / 20.0))
# max number of nearest neighbor iters to perform
n_iters = max(5, int(round(np.log2(X.shape[0]))))
# distance metric
metric="euclidean"
# number of neighbors for computing k-neighbor graph
n_neighbors = 10

# get nearest neighbors
nnd = NNDescent(
    X.reshape((len(X), np.product(np.shape(X)[1:]))),
    n_neighbors=n_neighbors,
    metric=metric,
    n_trees=n_trees,
    n_iters=n_iters,
    max_candidates=60,
    verbose=True
)
# get indices and distances
knn_indices, knn_dists = nnd.neighbor_graph

Tue Jan 26 17:16:57 2021 Building RP forest with 17 trees
Tue Jan 26 17:17:01 2021 NN descent for 16 iterations
	 1  /  16
	 2  /  16
	 3  /  16
	 4  /  16
	 5  /  16
	Stopping threshold met -- exiting after 5 iterations


In [9]:
# nearest neighbors and distances for each point in the dataset
np.shape(knn_indices), np.shape(knn_dists)

((60000, 10), (60000, 10))

#### Build fuzzy simplicial complex
+ The fuzzy_simplicial_set function takes the nearest neighbor graph and computes a graph of the probabilities of an edge exists between points.

In [11]:
# get indices and distances
knn_indices, knn_dists = nnd.neighbor_graph
random_state = check_random_state(None)

In [12]:
# build fuzzy_simplicial_set
umap_graph, sigmas, rhos = fuzzy_simplicial_set(
    X = X,
    n_neighbors = n_neighbors,
    metric = metric,
    random_state = random_state,
    knn_indices= knn_indices,
    knn_dists = knn_dists,
)

In [13]:
umap_graph

<60000x60000 sparse matrix of type '<class 'numpy.float32'>'
	with 803674 stored elements in Compressed Sparse Row format>

## VISUALIZATION STUFF

In [ ]:
# Node, Edge colors

edge_width = 0.1
edge_color = 'lightgrey'

edge_col = 'lightgrey'
edge_colordark = 'dimgrey'
opacity_edges = 0.5

opacity_nodes = 1.0
node_edge_col = None


# Node sizes 

#size = 10.0
#size3d = 5.0

# if node size reflects degree : 

scalef= 0.25
size = list(draw_node_degree(G, scalef).values())

scalef= 0.05
size3d = list(draw_node_degree_3D(G, scalef).values())

In [ ]:
# Centrality metrics 
color_method = 'clos'

df_centralities = pd.read_csv('output_csv/Features_centralities_Dataframe_'+organism+'.csv', index_col=0)

d_deghubs = dict(zip(G.nodes(),df_centralities['degs']))
d_clos = dict(zip(G.nodes(), df_centralities['clos']))
d_betw = dict(zip(G.nodes(), df_centralities['betw']))
d_eigen = dict(zip(G.nodes(), df_centralities['eigen']))

# uncomment to set colours
colours = color_nodes_from_dict(G, d_clos, color_method)

In [ ]:
def embed_umap_parametric_2D(Matrix, n_neighbors, spread, min_dist, metric='cosine'):
    n_components = 2 # for 2D

    U = ParametricUMAP(
        n_neighbors = n_neighbors,
        spread = spread,
        min_dist = min_dist,
        n_components = n_components,
        metric = metric)
    embed = U.fit_transform(Matrix)
    
    return embed

In [ ]:
%%time

umapparam_2D = embed_umap_parametric_2D(DM, n_neighbors, spread, min_dist)

In [ ]:
%%time 

posG_umapparam = get_posG_2D(genes, umapparam_2D)
df_posGparam = pd.DataFrame(posG_umapparam).T